In [3]:
import numpy as np
import pandas as pd
import datetime

# work_time_table = pd.read_csv('./근무시간_테이블.csv')

In [131]:
class working_time_calculator:
    def __init__(self):
#         self.work_time_table = pd.read_csv(file_path)
        self.work_time_table = pd.DataFrame(columns = ['date', 'month', 'week_num', 'week_day', 'start_time', 'end_time', 'spend_time', 'rest_time'])
        self.rest_time = 1
        
        self.days_dic = {
                            1: '월요일',
                            2: '화요일',
                            3: '수요일',
                            4: '목요일',
                            5: '금요일',
                            6: '토요일',
                            7: '일요일'
        }
        
        return
    
    # 현재 시점 기준으로 업무 시작, 종료를 입력하는 함수
    def work_start(self, start_time = None):
        if start_time == None:
            start_time = datetime.datetime.now()
            
        else:
            start_time = datetime.datetime.fromisoformat(start_time)
            
        date = datetime.datetime.strftime(start_time, '%Y%m%d')
        month = datetime.datetime.strftime(start_time, '%Y%m')
        week_num = start_time.isocalendar()[1]
        week_day = self.days_dic[start_time.isoweekday()]
        start_time = datetime.datetime.strftime(start_time, '%Y-%m-%d %H:%M')
        
        # 해당 일자의 데이터가 있을 경우, 없을 경우로 구분
        # 없을 경우 - 신규 데이터 입력
        # 없을 경우 - 기존 출근 시간을 수정
        update_state = not self.work_time_table.loc[self.work_time_table['date'] == date].empty
        
        if update_state:
            update_row = pd.DataFrame([[date, month, week_num, week_day, start_time]],
                                      columns = ['date', 'month', 'week_num', 'week_day', 'start_time'])
            self.work_time_table.where(self.work_time_table['date'] == date).update(update_row)
            
            # end_time까지 입력되어 있는 경우라면 spend_time도 다시 계산해주어야 한다.
            if self.work_time_table.where(self.work_time_table['date'] == date)['end_time'].notnull()[0]:
                self.get_spend_time(date)
        
        else:
            self.work_time_table.loc[len(self.work_time_table)] =\
            [date, month, week_num, week_day, start_time, np.nan, np.nan, self.rest_time]
        
        return
        
    def work_end(self, end_time = None):
        if end_time == None:
            end_time = datetime.datetime.now()
        
        else:
            end_time = datetime.datetime.fromisoformat(end_time)
        
        # end_time이 24시를 넘겼을 경우에 대한 고려가 필요
        date = datetime.datetime.strftime(end_time, '%Y%m%d')
        end_time = datetime.datetime.strftime(end_time, '%Y-%m-%d %H:%M')
        
        # input
        self.work_time_table.loc[self.work_time_table['date'] == date, 'end_time'] = end_time
        
        # spend_time 계산
        self.get_spend_time(date)
        
        return

    def get_spend_time(self, date):
        start_time, end_time = self.work_time_table.loc[self.work_time_table['date'] == date][['start_time', 'end_time']].values[0]
        
        start_time = datetime.datetime.fromisoformat(start_time)
        end_time = datetime.datetime.fromisoformat(end_time)
        
        spend_time = end_time - start_time - datetime.timedelta(hours = 1)
        spend_time = spend_time.total_seconds()
        
        # 한시간을 안넘을 경우, - 값이 나와서 값이 정확하게 안나옴
        if spend_time >= 0:
            spend_hours = int(spend_time // 3600)
            spend_minutes = int((spend_time % 3600) // 60)

            spend_time = f'{spend_hours:02d}:{spend_minutes:02d}'
        
        else:
            spend_hours = int(abs(spend_time) // 3600)
            spend_minutes = int((abs(spend_time) % 3600) // 60)
            
            spend_time = f'-{spend_hours:02d}:{spend_minutes:02d}'
        
        # input
        self.work_time_table.loc[self.work_time_table['date'] == date, 'spend_time'] = spend_time
        
        return
        
    def get_end_time(self, plan_time):
        
        finish_time_h = ''
        finish_time_m = ''
        print(f"오늘의 퇴근 예정 시간은 {int(finish_time_h):02d}:{int(finish_time_m)} 입니다.")
        return
    
    

In [2]:
start_time = '09:42'
time_plan = '6:13'

start_hour = int(start_time.split(':')[0])
start_minute = int(start_time.split(':')[1])
start_time = datetime.timedelta(
                                hours = start_hour,
                                minutes = start_minute
)

plan_hour = int(time_plan.split(':')[0])
plan_minute = int(time_plan.split(':')[1])
plan_time = datetime.timedelta(
                                hours = plan_hour,
                                minutes = plan_minute
)

break_time = datetime.timedelta(hours = 1)

finish_time = start_time + plan_time + break_time
finish_time_h = finish_time.total_seconds() // 3600
finish_time_m = (finish_time.total_seconds() % 3600) // 60
print(f"오늘의 퇴근 예정 시간은 {int(finish_time_h):02d}:{int(finish_time_m)} 입니다.")

오늘의 퇴근 예정 시간은 16:55 입니다.
